In [1]:
import sys
sys.path.append('/home/jp/Documents/GitProjects/faas-sim')
from typing import List

import matplotlib.pyplot as plt
from pandas import DataFrame
import pandas as pd
import numpy as np
import pickle
import datetime as dt
import matplotlib.dates as md
from matplotlib import ticker
import re

from ext.jjnp21.automator.experiment import Result
from ext.jjnp21.automator.analyzer import BasicResultAnalyzer



def load_results(path) -> List[Result]:
    with open(path, 'rb') as f:
        results = pickle.load(f)
        return results

In [2]:
results = load_results('/home/jp/Documents/tmp/optimization_aggressiveness.dump')

In [3]:
def cut_to_latter_half(result):
    count = len(result.invocations)
    result.invocations = result.invocations[int(count/2):]
    return result

In [4]:
cut_results = map(lambda r: cut_to_latter_half(r), results)
analyzer = BasicResultAnalyzer(cut_results)
analysis = analyzer.basic_kpis()

In [5]:
analysis

,name,E2E mean,E2E q50,E2E q75,E2E q90,E2E q99,FET mean,FET q50,FET q75,FET q90,...,tx_time_lb_fx mean,tx_time_lb_fx q50,tx_time_lb_fx q75,tx_time_lb_fx q90,tx_time_lb_fx q99,cl_fx_cross_city,cl_lb_fx_cross_city,nodes_by_type,total_rq_by_type,avg_rq_by_type
0,75rps_2000s_global_osmotic pthres=0.08 hyst=0.04,0.1521,0.1288,0.1709,0.2293,0.7351,0.0297,0.0170,0.0270,0.0952,...,0.0298,0.0098,0.0107,0.0888,0.6218,0.017775,0.017775,"{'rpi3': 17, 'rpi4': 16, 'tx2': 24, 'nuc': 25,...","{'rpi3': 5324, 'rpi4': 8091, 'tx2': 16869, 'nu...","{'rpi3': 313.1764705882353, 'rpi4': 505.6875, ..."
1,75rps_2000s_global_osmotic pthres=0.06 hyst=0.03,0.1496,0.1287,0.1630,0.2222,0.7300,0.0289,0.0171,0.0271,0.0939,...,0.0283,0.0098,0.0107,0.0871,0.6217,0.014642,0.014642,"{'rpi3': 13, 'rpi4': 18, 'tx2': 24, 'nuc': 27,...","{'rpi3': 4670, 'rpi4': 9131, 'tx2': 17240, 'nu...","{'rpi3': 359.2307692307692, 'rpi4': 507.277777..."
2,75rps_2000s_global_osmotic pthres=0.04 hyst=0.02,0.1539,0.1288,0.1694,0.2279,0.7441,0.0288,0.0169,0.0270,0.0954,...,0.0326,0.0098,0.0109,0.0902,0.6222,0.021570,0.021570,"{'rpi3': 14, 'rpi4': 15, 'tx2': 23, 'nuc': 24,...","{'rpi3': 4754, 'rpi4': 9056, 'tx2': 14957, 'nu...","{'rpi3': 339.57142857142856, 'rpi4': 603.73333..."
3,75rps_2000s_global_osmotic pthres=0.1 hyst=0.05,0.1553,0.1297,0.1689,0.2315,0.7427,0.0291,0.0169,0.0270,0.0940,...,0.0332,0.0098,0.0111,0.0905,0.6220,0.030756,0.030756,"{'rpi3': 16, 'rpi4': 17, 'tx2': 23, 'nuc': 24,...","{'rpi3': 5196, 'rpi4': 8734, 'tx2': 13600, 'nu...","{'rpi3': 324.75, 'rpi4': 513.7647058823529, 't..."
4,75rps_2000s_global_osmotic pthres=0.02 hyst=0.035,0.1586,0.1294,0.1790,0.2413,0.7614,0.0323,0.0172,0.0272,0.0958,...,0.0356,0.0099,0.0109,0.0917,0.6224,0.025043,0.025043,"{'rpi3': 15, 'rpi4': 16, 'tx2': 24, 'nuc': 24,...","{'rpi3': 4600, 'rpi4': 9071, 'tx2': 15931, 'nu...","{'rpi3': 306.6666666666667, 'rpi4': 566.9375, ..."
5,75rps_2000s_global_osmotic pthres=0.02 hyst=0.02,0.1585,0.1291,0.1726,0.2375,0.7603,0.0317,0.0171,0.0271,0.0956,...,0.0358,0.0099,0.0108,0.0909,0.6230,0.029101,0.029101,"{'rpi3': 19, 'rpi4': 16, 'tx2': 24, 'nuc': 26,...","{'rpi3': 4990, 'rpi4': 8140, 'tx2': 17519, 'nu...","{'rpi3': 262.63157894736844, 'rpi4': 508.75, '..."
6,75rps_2000s_global_osmotic pthres=0.02 hyst=0.01,0.1708,0.1315,0.1912,0.2683,0.8318,0.0318,0.0173,0.0274,0.0959,...,0.0470,0.0099,0.0112,0.0999,0.7019,0.054924,0.054924,"{'rpi3': 13, 'rpi4': 15, 'tx2': 24, 'nuc': 22,...","{'rpi3': 5237, 'rpi4': 8816, 'tx2': 17249, 'nu...","{'rpi3': 402.84615384615387, 'rpi4': 587.73333..."
7,75rps_2000s_global_osmotic pthres=0.03 hyst=0.015,0.1576,0.1292,0.1715,0.2330,0.7567,0.0313,0.0174,0.0271,0.0955,...,0.0351,0.0098,0.0108,0.0900,0.6226,0.030010,0.030010,"{'rpi3': 18, 'rpi4': 15, 'tx2': 24, 'nuc': 24,...","{'rpi3': 5285, 'rpi4': 6508, 'tx2': 17960, 'nu...","{'rpi3': 293.6111111111111, 'rpi4': 433.866666..."
8,75rps_2000s_global_osmotic pthres=0.02 hyst=0.05,0.1343,0.1251,0.1458,0.1985,0.3069,0.0276,0.0169,0.0269,0.0869,...,0.0156,0.0096,0.0100,0.0107,0.1147,0.002289,0.002289,"{'rpi3': 15, 'rpi4': 15, 'tx2': 24, 'nuc': 25,...","{'rpi3': 4466, 'rpi4': 5499, 'tx2': 16678, 'nu...","{'rpi3': 297.73333333333335, 'rpi4': 366.6, 't..."


In [6]:
def kpis_by_expname(expname):
    return analysis[analysis['name'] == expname]

In [9]:
def build_results_table(results):
    rows = []
    for r in results:
        row = {}
        row['Experiment Name'] = r.experiment.name
        row['LB Replicas'] = r.lb_replica_count['running_lb_replicas'].iloc[-1].astype(int)
        row['Mean TRT'] = kpis_by_expname(r.experiment.name)['E2E mean'].iloc[0].astype(float)
        row['Mean FET'] = kpis_by_expname(r.experiment.name)['FET mean'].iloc[0].astype(float)
        row['Mean LB_FX'] = kpis_by_expname(r.experiment.name)['tx_time_lb_fx mean'].iloc[0].astype(float)
        rows.append(row)
    return pd.DataFrame(rows)

build_results_table(results)

,Experiment Name,LB Replicas,Mean TRT,Mean FET,Mean LB_FX
0,75rps_2000s_global_osmotic pthres=0.08 hyst=0.04,5,0.1521,0.0297,0.0298
1,75rps_2000s_global_osmotic pthres=0.06 hyst=0.03,5,0.1496,0.0289,0.0283
2,75rps_2000s_global_osmotic pthres=0.04 hyst=0.02,5,0.1539,0.0288,0.0326
3,75rps_2000s_global_osmotic pthres=0.1 hyst=0.05,4,0.1553,0.0291,0.0332
4,75rps_2000s_global_osmotic pthres=0.02 hyst=0.035,5,0.1586,0.0323,0.0356
5,75rps_2000s_global_osmotic pthres=0.02 hyst=0.02,5,0.1585,0.0317,0.0358
6,75rps_2000s_global_osmotic pthres=0.02 hyst=0.01,5,0.1708,0.0318,0.0470
7,75rps_2000s_global_osmotic pthres=0.03 hyst=0.015,5,0.1576,0.0313,0.0351
8,75rps_2000s_global_osmotic pthres=0.02 hyst=0.05,17,0.1343,0.0276,0.0156
